In [1]:
import os
import re
import sys
import json
import codecs
import locale
import logging
import requests
from enum import Enum
import datetime as dt
from typing import Union, Optional, Tuple, List, cast

from matplotlib import pyplot as plt    # type: ignore  
import matplotlib.gridspec as gridspec  # type: ignore 

import pandas as pd
import numpy as np

In [2]:
#----------------------------------------------------------------
# Configurations section
#----------------------------------------------------------------

temp_content_dir = os.path.join(os.sep, 'tmp')
images_path = os.path.join("..","images")

now = dt.datetime.now()
sample_date = now.strftime("%d/%m/%Y")

locale.setlocale(locale.LC_ALL, 'it_IT.UTF-8')

ok_statuses = [200, 201, 202]
data_file_path = os.path.join(os.sep,"Users","ERIZZAG5J","Work","jmeter")
data_file_name = "merge_file.jtl"
csv_data_file = os.path.join(data_file_path, data_file_name)

image_file_prefix = "Test-"

class TypeOfOutputChart(Enum):
    PDF = 1
    PNG = 2
    JPG = 3

output_chart_type = TypeOfOutputChart.JPG

In [3]:
df = pd.read_csv(csv_data_file
                ,sep='#')  
(df.shape[0],df.dtypes)

(1140226,
 timeStamp            int64
 elapsed              int64
 label               object
 responseCode         int64
 responseMessage     object
 threadName          object
 dataType            object
 success               bool
 failureMessage      object
 bytes                int64
 sentBytes            int64
 grpThreads           int64
 allThreads           int64
 URL                 object
 Filename           float64
 Latency              int64
 SampleCount          int64
 ErrorCount           int64
 Hostname            object
 IdleTime             int64
 Connect              int64
 asset_id             int64
 dtype: object)

In [10]:
mask = (df["elapsed"] > 1000) & (df["responseCode"] == 200)
df.loc[mask,:].shape

(21080, 22)

In [8]:
df.loc[:,["threadName","grpThreads","allThreads"]]

,threadName,grpThreads,allThreads
0,node-01 Manifest 1-1,1,1
1,node-01 Manifest 1-1,1,1
2,node-01 Manifest 1-1,1,1
3,node-01 Manifest 1-1,1,1
4,node-01 Manifest 1-1,1,1
...,...,...,...
1140221,node-02 Manifest 1-36,5,5
1140222,node-02 Manifest 1-30,4,4
1140223,node-02 Manifest 1-25,3,3
1140224,node-02 Manifest 1-8,2,2


In [9]:
df.loc[:,"threadName"].unique()

array(['node-01 Manifest 1-1', 'node-01 Manifest 1-2',
       'node-01 Manifest 1-3', 'node-01 Manifest 1-4',
       'node-01 Manifest 1-5', 'node-01 Manifest 1-6',
       'node-01 Manifest 1-7', 'node-01 Manifest 1-8',
       'node-01 Manifest 1-9', 'node-01 Manifest 1-10',
       'node-01 Manifest 1-11', 'node-01 Manifest 1-12',
       'node-01 Manifest 1-13', 'node-01 Manifest 1-14',
       'node-01 Manifest 1-15', 'node-01 Manifest 1-16',
       'node-01 Manifest 1-17', 'node-01 Manifest 1-18',
       'node-01 Manifest 1-19', 'node-01 Manifest 1-20',
       'node-01 Manifest 1-21', 'node-01 Manifest 1-22',
       'node-01 Manifest 1-23', 'node-01 Manifest 1-24',
       'node-01 Manifest 1-25', 'node-01 Manifest 1-26',
       'node-01 Manifest 1-27', 'node-01 Manifest 1-28',
       'node-01 Manifest 1-29', 'node-01 Manifest 1-30',
       'node-01 Manifest 1-31', 'node-01 Manifest 1-32',
       'node-01 Manifest 1-33', 'node-01 Manifest 1-34',
       'node-01 Manifest 1-35', 'node-01